# Implikationen unterschiedlicher Risikomanagement-Strategien auf das Wert-/Risikoprofil einer Gesamtposition

© Thomas Robert Holy 2019
<br>
Version 0.2.0

## Grundlegende Einstellungen:

Zunächst müssen die notwendigen Pakete (auch Module) importiert werden, damit auf diese zugegriffen werden kann. 

In [ ]:
import pandas as pd # Programmbibliothek die Hilfsmittel für die Verwaltung von Daten und deren Analyse anbietet
import scipy.stats as stats # SciPy ist ein Python-basiertes Ökosystem für Open-Source-Software für Mathematik, Naturwissenschaften und Ingenieurwissenschaften
from scipy.stats import rankdata, norm  
from scipy import array, linalg, dot
import random # Dieses modul wird verwendet um Zufallszahlen zu ziehen
import numpy as np # Programmbibliothek die eine einfache Handhabung von Vektoren, Matrizen oder generell großen mehrdimensionalen Arrays ermöglicht
import math # Dieses Modul wird verwendet um Skalardaten zu berechnen, z. B. trigonometrische Berechnungen.
import operator # Programmbibliothek, welche die Ausgaben übersichtlicher gestaltet
import matplotlib.pyplot as plt # Programmbibliothek die es erlaubt mathematische Darstellungen aller Art anzufertigen
import matplotlib.patches as mpatches
import datetime as dt # Das datetime-Modul stellt Klassen bereit, mit denen Datums- und Uhrzeitangaben auf einfache und komplexe Weise bearbeitet werden können
import random # Dieses Modul implementiert Pseudozufallszahlengeneratoren für verschiedene Verteilungen.
import riskmeasure_module as rm

Anschließend werden Einstellungen definiert, die die Formatierung der Ausgaben betreffen.
Hierfür wird das Modul `operator` genutzt.
Außerdem wird die Größe der Grafiken modifiziert, welche später angezeigt werden sollen.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

SCREEN_WIDTH = 140 
centered = operator.methodcaller('center', SCREEN_WIDTH) 
pd.set_option('display.width', 125) 
plt.rcParams["figure.figsize"] = 15,12.5 

## Datensätze einlesen und manipulieren:

Nun werden Datensätze eingelesen und manipuliert.
Da Jupyter Notebook leider Eingaben nicht zeilenweise einlesen kann, müssen die Datensätze manuell definiert und anschließend zum Array "dateinamen" hinzufügt werden.
Standardmäßig werden fünf Datensätze (example1, ..., example5) definiert und im Array "dateinamen" gespeichert.
<br>
Anschließend wird aus jedem eingelesen Datensatz der Aktienkurs zum jeweiligen Tag extrahiert werden. 
Dieser Schritt wird automatisiert, indem zunächst die leere Liste "kurse" anlegt wird und anschließend von jedem sich in der Liste "dateinamen" befindenden Eintrag die jeweiligen Spalten "Date" und "Adj Close" eingelesen werden.
Dabei werden die verschiedenen im Datensatz vorhanden Spalten mit jedem Komma separiert und Punkte werden als Zeichen für die Dezimaltrennung interpretiert. Anschließend werden die so extrahierten Daten zum Array "kurse" hinzugefügt.
<br>
Danach wird das Modul `datetime` genutzt, um die Datumsspalte des jeweiligen Datensatzes bearbeitbar zu machen.
Zudem wird dem Programm mitgeteilt, dass die Einträge der Spalte "Adj Close" numerisch sind und mit ihnen gerechnet werden kann. Kommt es dabei zu Fehlern werden die entsprechende Werte als NaN-Werte behandelt.
<br><br>
Hinweis: An dieser Stelle können alternativ "richtige" Datensätze eingelesen werden, indem z.B. "example1" in BAS.DE usw. umbenannt werden, sofern die entsprechenden Datensätze im Home-Verzeichnis hochgeladen wurden.

In [ ]:
##########################################################################
#-------------------------------------------------------------------------
"""
datensatz1 = 'example1'
datensatz2 = 'example2'
datensatz3 = 'example3'
datensatz4 = 'example4'
datensatz5 = 'example5'
"""
datensatz1 = 'BAS.DE'
datensatz2 = 'FME.DE'
datensatz3 = 'NSU.DE'
datensatz4 = 'SIE.DE'
datensatz5 = 'VOW3.DE'

dateinamen = [datensatz1,datensatz2,datensatz3,datensatz4,datensatz5]

#-------------------------------------------------------------------------
##########################################################################

kurse = []
for eintrag in dateinamen:
    kurs = pd.read_csv(str(eintrag) + '.csv',
                decimal='.',
                usecols=['Date','Adj Close'])
    kurse.append(kurs) 

for eintrag in kurse:
    eintrag['Date'] = pd.to_datetime(eintrag['Date']) 
    eintrag['Adj Close'] = pd.to_numeric(eintrag['Adj Close'], errors='coerce') 

Nun werden zwei verschiedene DataFrames erzeugt, wobei "kurschart_0" die Basis für das Sharpe-Portfolio und "kurschart_1" die Basis für die Naive Diversifikation darstellt.
Beiden beinhalten die täglichen aus den eingelesenen Aktienkursen berechneten Renditen, wobei für "kurschart_1" direkt die Portfolio-Rendite mittels Naiver Diversifikation ermittelt wird.
Beide DataFrames sind auf die Handelstage eines Jahres beschränkt.

In [ ]:
kurschart_0 = pd.DataFrame()
kurschart_1 = pd.DataFrame()

zaehler = 0
for eintrag in kurse:
    x = dateinamen[zaehler]
    kurschart_0['Aktienkurs ' + str(x)] = eintrag['Adj Close']
    kurschart_1['Aktienkurs ' + str(x)] = eintrag['Adj Close']    
    zaehler += 1

kurschart_0 = kurschart_0[:252]
kurschart_0 = kurschart_0.pct_change()
kurschart_1 = kurschart_1[:252]
kurschart_1 = kurschart_1.pct_change()

#-------------------------------------------------------------------------
# Naive Diversifikation 

kurschart_1['PF-Rendite'] = (kurschart_1.sum(axis = 1, skipna = True) / len(dateinamen))

## Monte-Carlo-Simulation: Sharpe PF, efficient Frontier

Source of efficient frontier: https://medium.com/python-data/effient-frontier-in-python-34b0c3043314, https://medium.com/python-data/efficient-frontier-portfolio-optimization-with-python-part-2-2-2fe23413ad94

In [ ]:
trading_days = 253

# calculate daily and annual returns of the stocks
returns_daily = kurschart_0
returns_annual = returns_daily.mean() * trading_days

# get daily and covariance of returns of the stock
cov_daily = returns_daily.cov()
cov_annual = cov_daily * trading_days

port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

# set the number of combinations for imaginary portfolios
num_assets = len(dateinamen)
num_portfolios = 10000

#-------------------------------------------------------------------------

# populate the empty lists with each portfolios returns,risk and weights

for single_portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    weights /= np.sum(weights)
    returns = np.dot(weights, returns_annual)
    volatility = np.sqrt(np.dot(weights.T, np.dot(cov_annual, weights)))
    sharpe = returns / volatility
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

#-------------------------------------------------------------------------

# a dictionary for Returns and Risk values of each portfolio
portfolio = {'Returns': port_returns,
             'Volatility': port_volatility,
             'Sharpe Ratio': sharpe_ratio}

#-------------------------------------------------------------------------

# extend original dictionary to accomodate each ticker and weight in the portfolio
for counter, symbol in enumerate(dateinamen):
    portfolio[symbol + ' Weight'] = [Weight[counter] for Weight in stock_weights]

# make a nice dataframe of the extended dictionary
df = pd.DataFrame(portfolio)

# get better labels for desired arrangement of columns
column_order = ['Returns', 'Volatility', 'Sharpe Ratio'] + [stock + ' Weight' for stock in dateinamen]

# reorder dataframe columns
df = df[column_order]

#-------------------------------------------------------------------------

# find min Volatility & max sharpe values in the dataframe (df)
min_volatility = df['Volatility'].min()
max_sharpe = df['Sharpe Ratio'].max()

# use the min, max values to locate and create the two special portfolios
sharpe_portfolio = df.loc[df['Sharpe Ratio'] == max_sharpe]
min_variance_port = df.loc[df['Volatility'] == min_volatility]

# plot frontier, max sharpe & min Volatility values with a scatterplot
plt.style.use('seaborn-dark')
df.plot.scatter(x='Volatility', y='Returns', c='Sharpe Ratio',
                cmap='RdYlGn', edgecolors='black', figsize=(10, 8), grid=True)
plt.scatter(x=sharpe_portfolio['Volatility'], y=sharpe_portfolio['Returns'], c='red', marker='D', s=200)
plt.scatter(x=min_variance_port['Volatility'], y=min_variance_port['Returns'], c='blue', marker='D', s=200 )
plt.xlabel('Volatility (Std. Deviation)')
plt.ylabel('Expected Returns')
plt.title('Efficient Frontier')
plt.show()

#-------------------------------------------------------------------------

# print the details of the 2 special portfolios
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('Minimum-Varianz-Portfolio: \n' + str(min_variance_port.T))
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('Optimales Sharpe-Portfolio: \n' + str(sharpe_portfolio.T))
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

## Optimales Sharpe-Portfolio und Datenbereinigung

In diesem Schritt werden die durch die Monte-Carlo-Simulation ermittelten optimalen Gewichte des Sharpe-Portfolios auf den DataFrame "kurschart_0" angewendet, sodass im Anschluss die Portfolio-Rendite errechnet werden kann.
Danach werden die Portfolio-Renditen in einer Liste gespeichert und bereinigt.

In [ ]:
#-------------------------------------------------------------------------
# Optimales Sharpe-Portfolio

sharpe_portfolio = sharpe_portfolio.values.tolist()
sharpe_portfolio = sharpe_portfolio[0][3::]

kurschart_0 = kurschart_0.multiply(sharpe_portfolio, axis = 1)
kurschart_0['PF-Rendite'] = kurschart_0.sum(axis = 1, skipna = True)

#-------------------------------------------------------------------------
# Datenbereinigung

sharpe_values_PF = kurschart_0['PF-Rendite'].values.tolist()
sharpe_values_PF = np.array(sharpe_values_PF)
sharpe_values_PF = sharpe_values_PF[np.logical_not(np.isnan(sharpe_values_PF))]
sharpe_values_PF = sharpe_values_PF[sharpe_values_PF != 0.0]

#----------------------------------

naiv_values_PF = kurschart_1['PF-Rendite'].values.tolist()
naiv_values_PF = np.array(naiv_values_PF)
naiv_values_PF = naiv_values_PF[np.logical_not(np.isnan(naiv_values_PF))]
naiv_values_PF = naiv_values_PF[naiv_values_PF != 0.0]

## Auswertung des Sharpe-Portfolios

### Funktionen Definieren und Verteilungsfunktionen plotten

In diesem Abschnitt werden zunächst Mittelwert und Standardabweichung des Sharpe-Portfolios berechnet und ausgegeben. 
Anschließend werden Funktionen definiert, welche das Plotten der Verteilungsfunktionen für die historische Simulation und die Varianz-Kovarianz-Methode und die Ermittlung der Risikomaße auf Basis des jeweiligen Simulationsverfahrens vereinfachen.

In [ ]:
print('##############################################################################################################################################')
print('|' + centered('Optimales PF nach Sharpe') + '| ')
print('##############################################################################################################################################')

#-------------------------------------------------------------------------

mu_sharpe_PF = np.mean(sharpe_values_PF)
std_sharpe_PF = np.std(sharpe_values_PF)

print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('[INFO] Die Porfolio-Rendite hat einen Erwartunswert i.H.v. ' + str(mu_sharpe_PF) + '.') + '| ')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('[INFO] Das Porfolio hat eine Standardabweichung i.H.v. ' + str(std_sharpe_PF) + '.') + '| ')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

mini_values_PF = min(min(naiv_values_PF), min(sharpe_values_PF))
maxi_values_PF = max(max(naiv_values_PF), max(sharpe_values_PF)) 

bins = len(naiv_values_PF)

#-------------------------------------------------------------------------
# Historische Simulation - Plot

def hist_sim(values, bins):
    H, X1 = np.histogram(values, bins, density=True)
    dx = X1[1] - X1[0]
    F1 = np.cumsum(H) * dx 
    plt.plot(X1[1:], F1) 
    
hist_sim(sharpe_values_PF, bins)

#-------------------------------------------------------------------------
# Varianz-Kovarianz-Methode - Plot

def var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu, std):
    global var_covar_results
    
    array = np.array(np.arange(0.0001, 1, 0.0001))
    var_covar_results = stats.norm.ppf(array, mu, std)

    var_covar_range = np.linspace(mini_values_PF, maxi_values_PF, bins)
    plt.plot(var_covar_range, stats.norm.cdf(var_covar_range, mu, std))
    
var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu_sharpe_PF, std_sharpe_PF)

#-------------------------------------------------------------------------
# Restliche Einstellungen für die Grafik

def easy_plot():
    plt.xlabel('Rendite') 
    plt.ylabel('Wahrscheinlichkeit') 
    blue_patch = mpatches.Patch(color='blue', label='Historische Simulation') 
    orange_patch = mpatches.Patch(color='orange', label='Varianz-Kovarianzmethode') 
    plt.legend(handles=[orange_patch, blue_patch]) 
    plt.title('Verteilungsfunktion: Historische Simulation versus Varianz-Kovarianz-Methode')
    plt.grid() 
    plt.axhline(0, color='black') 
    plt.axvline(0, color='black') 
    plt.show() 
easy_plot() 

#-------------------------------------------------------------------------
# Risikomessung - Historische Simulation

RM_list = []
def risk(values, alpha, gamma):
    global RM_list
    
    alle_RM =  rm.risk_measure()
    alle_RM.get_all(values, alpha, gamma)
    VaR, CVaR, Power_Risk, Power_EW = alle_RM.all
    
    RM_list.append([VaR, CVaR, Power_Risk, Power_EW])

    print('|' + centered('Der VaR beträgt: ' + str(VaR) + '.') + '| ')
    print('|' + centered('Der CVaR beträgt: ' + str(CVaR) + '.') + '| ')
    print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
    print('|' + centered('Power-Spektrales Risikomaß:') + '| ')
    print('|' + centered('Der Erwartungswert beträgt: ' + str(Power_EW) + '.') + '| ')
    print('|' + centered('Das Risiko beträgt: ' + str(Power_Risk) + '.') + '| ')

### Risikomaße schätzen - Parameterfestlegung und Aufruf der Funktionen

In [ ]:
##########################################################################
#-------------------------------------------------------------------------

alpha = 0.1
gamma = 0.5

#-------------------------------------------------------------------------
##########################################################################

print('##############################################################################################################################################')
print('|' + centered('Sharpe: Risikomessung - Historische Simulation') + '| ')
print('##############################################################################################################################################')

risk(sharpe_values_PF, alpha, gamma)

#-------------------------------------------------------------------------
# Risikomessung - Varianz-Kovarianz-Methode

print('##############################################################################################################################################')
print('|' + centered('Sharpe: Risikomessung - Varianz-Kovarianz-Methode') + '| ')
print('##############################################################################################################################################')

risk(var_covar_results, alpha, gamma)
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

## Auswertung des naiv diversifizierten Portfolios

Da die Funktionen zum Plotten der Verteilungsfunktionen und zur Risikomessung im vorherigen Schritt bereits definiert wurden, müssen diese hier nur noch mit den entsprechenden Daten der naiven Diversifikation aufgerufen werden.

In [ ]:
print('##############################################################################################################################################')
print('|' + centered('Naive Diversifikation') + '| ')
print('##############################################################################################################################################')

#-------------------------------------------------------------------------

mu_naiv_PF = np.mean(naiv_values_PF)
std_naiv_PF = np.std(naiv_values_PF)

print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('[INFO] Die Porfolio-Rendite hat einen Erwartunswert i.H.v. ' + str(mu_naiv_PF) + '.') + '| ')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('[INFO] Das Porfolio hat somit eine Standardabweichung i.H.v. ' + str(std_naiv_PF) + '.') + '| ')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

#-------------------------------------------------------------------------
# Historische Simulation - Plot

hist_sim(naiv_values_PF, bins)

#-------------------------------------------------------------------------
# Varianz-Kovarianz-Methode - Plot

var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu_naiv_PF, std_naiv_PF)

#-------------------------------------------------------------------------
# Restliche Einstellungen für die Grafik

easy_plot() 

#-------------------------------------------------------------------------
# Risikomessung - Historische Simulation

print('##############################################################################################################################################')
print('|' + centered('Sharpe: Risikomessung - Historische Simulation') + '| ')
print('##############################################################################################################################################')

risk(naiv_values_PF, alpha, gamma)

#-------------------------------------------------------------------------
# Risikomessung - Varianz-Kovarianz-Methode

print('##############################################################################################################################################')
print('|' + centered('Sharpe: Risikomessung - Varianz-Kovarianz-Methode') + '| ')
print('##############################################################################################################################################')

risk(naiv_values_PF, alpha, gamma)
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')

## Gegenüberstellung der Verteilungsfunktionen und Risikomaße

Um die Verteilungsfunktionen und die berechneten Risikomaße besser vergleichen zu können, werden diese hier noch einmal zusammengetragen.

In [ ]:
#-------------------------------------------------------------------------
#  Sharpe-Portfolio: Historische Simulation - Plot (Blau)

hist_sim(sharpe_values_PF, bins)

#-------------------------------------------------------------------------
# Sharpe-Portfolio: Varianz-Kovarianz-Methode - Plot (Orange)

var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu_sharpe_PF, std_sharpe_PF)

#-------------------------------------------------------------------------
# Naive Diversifikation: Historische Simulation - Plot (Grün)

hist_sim(naiv_values_PF, bins)

#-------------------------------------------------------------------------
# Naive Diversifikation: Varianz-Kovarianz-Methode - Plot (Rot)

var_co_var_sim(mini_values_PF, maxi_values_PF, bins, mu_naiv_PF, std_naiv_PF)

#-------------------------------------------------------------------------
# Restliche Einstellungen für die Grafik

plt.xlabel('Rendite') 
plt.ylabel('Wahrscheinlichkeit') 
blue_patch = mpatches.Patch(color='blue', label='Sharpe - Historische Simulation') 
orange_patch = mpatches.Patch(color='orange', label='Sharpe - Varianz-Kovarianzmethode') 
green_patch = mpatches.Patch(color='green', label='Naiv - Historische Simulation') 
red_patch = mpatches.Patch(color='red', label='Naiv - Varianz-Kovarianzmethode')     
plt.legend(handles=[orange_patch, blue_patch, green_patch, red_patch]) 
plt.title('Verteilungsfunktion: Sharpe/Naiv - Historische Simulation versus Varianz-Kovarianz-Methode')
plt.grid() 
plt.axhline(0, color='black') 
plt.axvline(0, color='black') 
plt.show() 

#-------------------------------------------------------------------------
# DataFrame Risikomessung: Übersicht

RM_DataFrame = pd.DataFrame()
RM_DataFrame['Index'] = ['VaR', 'CVaR', 'P-SRM (Risk)', 'P-SRM (EW)']
RM_DataFrame['Sharpe + Historisch'] = RM_list[0]
RM_DataFrame['Naiv + Historisch'] = RM_list[2]
RM_DataFrame['Sharpe + VarKoVar'] = RM_list[1]
RM_DataFrame['Naiv + VarKoVar'] = RM_list[3]
RM_DataFrame = RM_DataFrame.set_index('Index')

print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print('|' + centered('Sharpe Portfolio versus Naive Diversifikation: Risikomessung') + '| ')
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')
print(RM_DataFrame)
print('#--------------------------------------------------------------------------------------------------------------------------------------------#')